In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('D:\Python_about\PYTHON接单4\优惠卷预测\ccf_offline_stage1_train.csv')

In [3]:
def process_data(df, label_name):
    df = df[~df['Coupon_id'].isnull()]
    df['Distance'] = df['Distance'].fillna(0)
    df['Label'] = df[label_name].apply(lambda x: 0 if pd.isna(x) or x == '' else 1)
    df[['x', 'y']] = df['Discount_rate'].str.split(':', expand=True)
    df['y'] = df['Distance'].fillna(0)
    df = df[['Distance','x','y','Label']]
    return df

In [4]:
df = process_data(df, 'Date')

In [5]:
df

,Distance,x,y,Label
1,1.0,150,1.0,0
2,0.0,20,0.0,0
3,0.0,20,0.0,0
4,0.0,20,0.0,0
6,0.0,20,0.0,1
...,...,...,...,...
1754877,6.0,30,6.0,1
1754878,2.0,30,2.0,1
1754880,6.0,30,6.0,1
1754882,6.0,50,6.0,0


In [6]:
# X_train = train.drop(['Label'],axis=1)
# y_train = train['Label']
# X_test = test.drop(['Label'],axis=1)
# y_test = test['Label']
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1]
y = df.iloc[:, -1]
class_counts = y.value_counts()
print(class_counts)
# 分割数据集，80% 训练数据和 20% 测试数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

0    977900
1     75382
Name: Label, dtype: int64


In [7]:
from sklearn import svm, tree, linear_model, ensemble
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score

# 初始化模型
models = {
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Linear Regression": linear_model.LogisticRegression(),  # 用逻辑回归代替线性回归进行分类
    "Random Forest": ensemble.RandomForestClassifier()
}

# 训练并评估模型
for name, model in models.items():
    # 训练模型
    model.fit(X_train, y_train)

    # 进行预测
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # 获取为正类的概率，用于计算 AUC

    # 计算评估指标
    acc = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    print(f"{name}:\n Accuracy: {acc:.2f}, Recall: {recall:.2f}, AUC: {auc:.2f}\n")


Decision Tree:
 Accuracy: 0.93, Recall: 0.00, AUC: 0.76

Linear Regression:
 Accuracy: 0.93, Recall: 0.00, AUC: 0.75

Random Forest:
 Accuracy: 0.93, Recall: 0.00, AUC: 0.76

